In [ ]:
import pandas as pd
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
import scipy.stats as stats
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import numpy as np
import os
import sys
from arch import arch_model
import matplotlib as mpl
from statsmodels.stats.diagnostic import acorr_ljungbox #白噪声检验
from statsmodels.graphics.tsaplots import plot_predict
from sklearn.metrics import mean_squared_error
#######读取数据等
from statsmodels.tsa.stattools import adfuller
import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")
# 设置字体为支持中文的字体
font = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf", size=14)
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [ ]:
data=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/merged_data_123.xlsx")

In [ ]:
asd=pd.read_excel("C:/Users/韩甄泽/OneDrive/桌面/2023年秋数据科学导论大作业/附件3_项目三数据集/附件4.xlsx",sheet_name='Sheet1')

In [ ]:
data

In [ ]:
data['销售日期']=pd.to_datetime(data['销售日期'])
data = data.groupby(['销售日期', '单品名称']).agg({
    '销量(千克)': 'sum',
    '销售单价(元/千克)': lambda x: (x * data.loc[x.index, '销量(千克)']).sum() / data.loc[x.index, '销量(千克)'].sum(),
    '批发价格(元/千克)': lambda x: (x * data.loc[x.index, '销量(千克)']).sum() / data.loc[x.index, '销量(千克)'].sum()
}).reset_index()
data1=data[((data['销售日期']>='2023-06-24')&(data['销售日期']<='2023-06-30'))]
data2=data1[data1['销量(千克)']>=2.5]


In [ ]:
selected_products = []
start_date = datetime.date(2023, 6, 24)
end_date = datetime.date(2023, 6, 30)
for day in range((end_date - start_date).days + 1):
    current_date = start_date + datetime.timedelta(days=day)
    current_date_str = current_date.strftime('%Y-%m-%d')
    daily_products = data2[(data2['销售日期'] == current_date_str) & (data2['销量(千克)'] > 2.5)]['单品名称'].tolist()
    if len(daily_products) > 0:
        selected_products.extend(daily_products)
        

result = []
for product in set(selected_products):
    if selected_products.count(product) >= 3:
        result.append(product)



In [ ]:
# 创建一个空的数据框，用于存放加权平均值结果
weighted_avg_df = pd.DataFrame(columns=['单品名称', '加权平均销量'])
for product in result:
    product_data=data2[data2['单品名称']==product]
    flitered_data =product_data[(product_data['销售日期']>=start_date.strftime('%Y-%m-%d'))&(product_data['销售日期']<=end_date.strftime('%Y-%m-%d'))]
    weighted_sum = sum([price * weight for price, weight in zip(flitered_data['批发价格(元/千克)'],flitered_data['销量(千克)'] )])
    #计算加权平均值
    total_mean=weighted_sum/sum(flitered_data['销量(千克)'])
    weighted_avg_df=weighted_avg_df.append({'单品名称':product,'加权平均值':total_mean},ignore_index=True)

weighted_avg_df2=weighted_avg_df[['单品名称','加权平均值']].copy()
weighted_avg_df2.to_excel("C:/Users/韩甄泽/OneDrive/桌面/s1234.xlsx")

In [ ]:
# 定义函数来检测和删除异常值
def remove_outliers(df, column, threshold=3):
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    df_no_outliers = df[z_scores < threshold]
    return df_no_outliers
result_df=pd.DataFrame(columns=['单品名称', '系数k', '截距b'])
# 按分类编码分组
grouped_df = data.groupby('单品名称')

# 要处理的单品码编列表
target_product_codes = [
    "云南生菜(份)", "娃娃菜", "姜蒜小米椒组合装(小份)", "西兰花", "长线茄", "双孢菇(盒)", "金针菇(盒)", "螺丝椒(份)",
    "红薯尖", "菠菜(份)", "小米椒(份)", "虫草花(份)", "枝江青梗散花", "海鲜菇(包)", "净藕(1)", "高瓜(1)",
    "小皱皮(份)", "青茄子(1)", "西峡花菇(1)", "紫茄子(2)", "云南油麦菜(份)", "木耳菜", "上海青", "奶白菜",
    "螺丝椒", "竹叶菜", "小青菜(1)", "洪湖藕带", "芜湖青椒(1)", "苋菜"
]
# 循环遍历每个商品的相关系数
for code, group_df in grouped_df:
    max_corr = 0
    max_r2 = -1
    best_code = None
    best_date_range = None
    if code in target_product_codes:
        
        group_df = group_df[group_df['销售类型'] == '销售']
        group_df = group_df[group_df['是否打折销售'] == '否']
        group_df = group_df.groupby(['销售日期', '单品名称']).agg({
        '销量(千克)': 'sum',
        '销售单价(元/千克)': lambda x: (x * group_df.loc[x.index, '销量(千克)']).sum() / group_df.loc[x.index, '销量(千克)'].sum(),
        '批发价格(元/千克)': lambda x: (x * group_df.loc[x.index, '销量(千克)']).sum() / group_df.loc[x.index, '销量(千克)'].sum()
        
    }).reset_index()
        # 计算相关系数
        correlation_coefficient = group_df['批发价格(元/千克)'].corr(group_df['销售单价(元/千克)'])
        
        features = group_df[['批发价格(元/千克)']]
        target = group_df['销售单价(元/千克)']

        # 将数据分为训练集和测试集
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=42)

        # 创建并训练线性回归模型
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train, y_train)

        # 在测试集上进行预测
        y_pred = model.predict(X_test)

        # 评估模型性能
        mse = mean_squared_error(y_test, y_pred)
        # 均方根误差（Root Mean Squared Error, RMSE）
        rmse = np.sqrt(mse)
        # 平均绝对误差（Mean Absolute Error, MAE）
        mae = mean_absolute_error(y_test, y_pred)
        # 决定系数（Coefficient of Determination, R^2）
        r2 = r2_score(y_test, y_pred)
    
        if abs(correlation_coefficient) > max_corr and abs(r2) > max_r2 and abs(r2) < 1:
            max_corr = abs(correlation_coefficient)
            max_r2 = abs(r2)

        
        k = model.coef_[0]
        result_df=result_df.append({'单品名称':code, '系数k':k-1, '截距b':model.intercept_},ignore_index=True)
        
        # 输出相关系数最高且 R^2 最大的单品名称和日期范围
        print(f"相关系数最高且 R^2 最大的单品名称: {code}")
        print(f"日期范围: {best_date_range}")
        print(f"相关系数: {max_corr}")
        print(f"R^2 值: {max_r2}")
        # 打印线性回归模型的系数和截距
        print(f'截距 (Intercept): {model.intercept_}')
        print(f'系数 (Coefficients): {model.coef_}')

        # 打印线性回归模型的公式
        formula = f'销售单价 = {model.intercept_:.4f} + '
        for i, coef in enumerate(model.coef_):
            formula += f'{coef:.4f} * {features.columns[i]}'
            if i < len(model.coef_) - 1:
                formula += ' + '

        print('线性回归公式:', formula)


In [ ]:
merged_df=pd.merge(result_df,asd[['单品名称','损耗率(%)']],on='单品名称')
merged_df.to_excel("C:/Users/韩甄泽/OneDrive/桌面/问题三/kb无截距.xlsx")